[View in Colaboratory](https://colab.research.google.com/github/hjjimmykim/prospectiveporpoise/blob/master/Untitled0.ipynb)

LOLA

Ideas:
1. RL vs. RL (multiagent)
2. Communication channel (pre-set? deception?)
3. LSTM stacked on FWN
4. Induce RL to learn ZD-strategy (learn probabilities?)
5. RL that adapts to opponent's non-stationary strategy

# Libraries

In [214]:
%load_ext autoreload
%autoreload 2

# Standard
import numpy as np
import matplotlib.pyplot as plt
import copy
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parameters

In [0]:
#Reward structure
T = 5 # Temptation
R = 3 # Reward
P = 1 # Penalty
S = 0 # Sucker

# Reward matrix (0 = cooperate, 1 = defect)
RM = np.zeros([2,2])
RM[0][0] = R
RM[0][1] = S
RM[1][0] = T
RM[1][1] = P

# RL
gamma = 0.3 # Decay rate
alpha = 0.005 # Learning rate

# Simulation
n_steps = 300      # Training steps
n_turns = 10       # Turns in the test game
n_rec = 50        # For printing purposes

# Models

In [0]:
# Converts action-pair [a1,a2] into a scalar state (0,1,2,3,4)
def action_to_state(a1,a2):
  
  if a1 == -1 and a2 == -1: # Initial
    s = 0
  elif a1 == 0 and a2 == 0: # CC
    s = 1
  elif a1 == 0 and a2 == 1: # CD
    s = 2
  elif a1 == 1 and a2 == 0: # DC
    s = 3
  elif a1 == 1 and a2 == 1: # DD
    s = 4
    
  return s

In [0]:
def mathematica2py(input = 'V.txt', ID_me=1, ID_otherguy=2):
    f = open(input)

    text = f.read()

    text = ''.join(text.split()) # Remove whitespace
    text = text.replace('^','**') # Replace ^ with **
    
    '''
    text = text.replace('a0','p' + str(ID_me) + '[0]')
    text = text.replace('a1','p' + str(ID_me) + '[1]')
    text = text.replace('a2','p' + str(ID_me) + '[2]')
    text = text.replace('a3','p' + str(ID_me) + '[3]')
    text = text.replace('a4','p' + str(ID_me) + '[4]')
    
    text = text.replace('b0','p' + str(ID_otherguy) + '[0]')
    text = text.replace('b1','p' + str(ID_otherguy) + '[1]')
    text = text.replace('b2','p' + str(ID_otherguy) + '[2]')
    text = text.replace('b3','p' + str(ID_otherguy) + '[3]')
    text = text.replace('b4','p' + str(ID_otherguy) + '[4]')
    '''

    num_terms = text.count('{')

    # Extract terms and get rid of curly braces
    terms = []
    while text.find('}') > -1:
        cut_point = text.find('}')
        terms.append(text[:cut_point].strip('{'))
        text = text[cut_point+1:]
        
    return terms

In [0]:
def evaluate_gradient(dV, a, b):
  # dV = output of mathematica2py
  # a = my policy
  # b = other guy's policy
  
  a0 = a[0]
  a1 = a[1]
  a2 = a[2]
  a3 = a[3]
  a4 = a[4]
  
  b0 = b[0]
  b1 = b[1]
  b2 = b[2]
  b3 = b[3]
  b4 = b[4]
  
  return eval(dV)

In [219]:
V = mathematica2py('V.txt') # Value function (5 elements for each state)

# Gradients
DVme = [] # w.r.t. my parameters (ai's)
for i in range(5):
  DVme.append(mathematica2py('DVa' + str(i) + '.txt'))
  
DVotherguy = [] # w.r.t. other guy's parameters (bi's)
for i in range(5):
  DVotherguy.append(mathematica2py('DVb' + str(i) + '.txt'))

p1 = [1,1,1,1,1]
# Win-stay-lose-shift
#p2 = [1,1,0,0,1]
# Tit-for-tat
p2 = [1,1,1,0,0]

# DVme[i][j] = Gradient of state j value function with respect to a_i
# DVotherguy[i][j] = Gradient of state j value function with respect to b_i

#print(evaluate_gradient(DVme[0][0], p1, p2))

print(p1)
print(p2)

[1, 1, 1, 1, 1]
[1, 1, 1, 0, 0]


# Game

In [220]:
t_start = time.time()
t1 = time.time()

# Gradient ascent
for i_ep in range(n_steps):
  for i_pol in range(5):
      for j_pol in range(5):
        p1[i_pol] = p1[i_pol] + alpha*evaluate_gradient(DVme[i_pol][j_pol],p1,p2)
        p1[i_pol] = min(1,p1[i_pol])
        p1[i_pol] = max(0,p1[i_pol])
        
  if i_ep % n_rec == 0 and i_ep != 0: 
    t2 = time.time()
    print('-----------------------------')
    print('Episode ' + str(i_ep))
    print(p1)
    print('Runtime for episodes ' + str(i_ep - n_rec) + '-' + str(i_ep) + ': ' + str(t2-t1) + ' s')
    print('-----------------------------')
    t1 = t2

# Test game

# Keep track of rewards
rewards1 = np.zeros([n_turns,1])
rewards2 = np.zeros([n_turns,1])

# Keep track of actions
actions1 = np.zeros([n_turns,1])
actions2 = np.zeros([n_turns,1])

# Initial state
s = 0

for i_turn in range(n_turns):

  # Choose action
  # 0 = cooperate, 1 = defect
  a1 = int(np.random.rand(1) > p1[s])
  a2 = int(np.random.rand(1) > p2[s])
  
  actions1[i_turn] = a1
  actions2[i_turn] = a2

  # Get rewards
  r1 = RM[a1][a2]
  r2 = RM[a2][a1]

  rewards1[i_turn] = r1
  rewards2[i_turn] = r2

  # Update the state
  s = action_to_state(a1,a2)

print(actions1)
print(actions2)
print(np.sum(rewards1))
print(np.sum(rewards2))
  
print('Total runtime: ' + str(time.time() - t_start) + ' s')

-----------------------------
Episode 50
[0.918473030829616, 0.8049002642175533, 0.867019606332027, 0.9936259099039279, 0.9942292875258619]
Runtime for episodes 0-50: 5.866816520690918 s
-----------------------------
-----------------------------
Episode 100
[0.8427770247682157, 0.6432555511623597, 0.7392199942117487, 0.9904277372849505, 0.99175195055795]
Runtime for episodes 50-100: 5.747097015380859 s
-----------------------------
-----------------------------
Episode 150
[0.7702362184506694, 0.5035390354389285, 0.6133014148902511, 0.9908179843982909, 0.9919970344798492]
Runtime for episodes 100-150: 5.766766309738159 s
-----------------------------
-----------------------------
Episode 200
[0.7001545357852003, 0.3811118467861153, 0.4886793321202194, 0.9951223289284558, 0.9946647748566267]
Runtime for episodes 150-200: 5.75872802734375 s
-----------------------------
-----------------------------
Episode 250
[0.6320265197080505, 0.27285305436525525, 0.3648375630362138, 1, 0.999524926

In [221]:
p1

[0.5667951250434425, 0.1784254256413493, 0.24403264267352265, 1, 1]

In [222]:
p2

[1, 1, 1, 0, 0]

In [223]:
s2

1

In [224]:
actions2

array([[0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.]])